In [31]:
import numpy as np
import statistics
import matplotlib.pyplot as plt
from scipy.stats import norm

class Trade:
    def __init__(self, S0_1, S0_2,q1, q2, K,sigma, r, rho, tol, T):
        #self.option = option
        self.S0_1 = S0_1 
        self.S0_2 = S0_2
        self.q1 = q1 
        self.q2 = q2
        self.K = K
        self.sigma = sigma 
        self.r = r 
        self.rho = rho 
        self.tol = tol 
        self.T = T
        #self.N = N
        
    def value(self):
        option = input('What option do you want to evaluate ("Call", "Put")')

        #Calculate theoretical option values
        F1 = self.S0_1*np.exp((self.r-self.q1)*self.T)
        F2 = self.S0_2*np.exp((self.r-self.q2)*self.T)

        sigma_k = np.sqrt(self.sigma**2-2*(F2/(F2+self.K))*self.rho*self.sigma*self.sigma+ (F2/(F2+self.K))**2*self.sigma**2)

        d1= (np.log(F1/(F2+self.K)) + (sigma_k**2/2)*self.T)/(sigma_k*np.sqrt(self.T))
        d2= d1-sigma_k*np.sqrt(self.T)

        #d1_2= (np.log(S0_2/K) + (r-q2+sigma**2/2)*T)/(sigma*np.sqrt(T))
        #d2_2= d1_2-sigma*np.sqrt(T)

        theo_value = 0

        if option =="Call":
            theo_value = np.exp(-self.r*self.T)*(F1*norm.cdf(d1) -(F2+self.K)*norm.cdf(d2))
        else: 
            theo_value = np.exp(-self.r*self.T)*(F1*norm.cdf(d1) -(F2+self.K)*norm.cdf(d2)) - np.exp(-self.r*self.T)*(F1-F2-self.K)

        print("Approximation of theoretical option value is", theo_value)

        k=10000
        #Calculate Payoff and sd of sample and estimate N

        mean=np.zeros(2)
        cov= [[1,self.rho],[self.rho,1]]
        z_sample = np.random.multivariate_normal(mean, cov, size = k)
        z1_sample = z_sample[:,0]
        z2_sample = z_sample[:,1]

        St_1_sample= self.S0_1*np.exp((self.r-self.q1-(self.sigma**2)/2)*self.T+self.sigma*np.sqrt(self.T)*z1_sample)
        St_2_sample= self.S0_2*np.exp((self.r-self.q2-(self.sigma**2)/2)*self.T+self.sigma*np.sqrt(self.T)*z2_sample)

        W_sample = np.zeros(k)

        if option =="Call":
            W_sample = np.exp(-self.r*self.T)*np.maximum(St_1_sample - St_2_sample -self.K, 0)
        else: 
            W_sample = np.exp(-self.r*self.T)*np.maximum(-St_1_sample+ St_2_sample +self.K, 0)

        sd_sample= np.std(W_sample)
        
        N=int((1.96*sd_sample/(self.tol/2))**2)
        
        #Performing simulations
        
        z = np.random.multivariate_normal(mean, cov, size = N)
        z1 = z[:,0]
        z2 = z[:,1]

        St_1= self.S0_1*np.exp((self.r-self.q1-(self.sigma**2)/2)*self.T+self.sigma*np.sqrt(self.T)*z1)
        St_2= self.S0_2*np.exp((self.r-self.q2-(self.sigma**2)/2)*self.T+self.sigma*np.sqrt(self.T)*z2)

        W = np.zeros(N)

        if option =="Call":
            #W1 is value of call
            W1 = np.exp(-self.r*self.T)*np.maximum(St_1 - St_2 -self.K, 0)
            #W2 is value of put in this case
            W2 = np.exp(-self.r*self.T)*np.maximum(-St_1+St_2+self.K, 0)
        else: 
            #W1 is value of put
            W1 = np.exp(-self.r*self.T)*np.maximum(-St_1+St_2+self.K, 0)
            #W2 is value of call in this case
            W2 = np.exp(-self.r*self.T)*np.maximum(St_1 - St_2 -self.K, 0)

        V_sim1 = np.mean(W1)
        V_sim2 = np.mean(W2)

        sd1 = np.std(W1)
        conf_int = np.zeros(2)
        conf_int[0] = V_sim1-1.96*sd1/np.sqrt(N)
        conf_int[1] = V_sim1+1.96*sd1/np.sqrt(N)

        #Verify put call parity: 
        if option =="Call":
            trade1 = V_sim1+ np.exp(-self.r*self.T)*self.K
            trade2 = V_sim2+np.exp(-self.q1*self.T)*self.S0_1- np.exp(-self.q2*self.T)*self.S0_2
            difference = trade1 - trade2
        else: 
            trade1 = V_sim2+ np.exp(-self.r*self.T)*self.K
            trade2 = V_sim1+np.exp(-self.q1*self.T)*self.S0_1- np.exp(-self.q2*self.T)*self.S0_2
            difference = trade1 - trade2
            
        print("Values for N, simulated option value, confidence interval, and difference between 2 trading strategies are\n")
        return N, V_sim1, conf_int, difference

    
a = Trade(180, 160, 0.01,0.015,20, 0.2, 0.02, 0.7, 0.1, 180/365)
a.value()

What option do you want to evaluate ("Call", "Put")Put
Approximation of theoretical option value is 7.1583427901533545
Values for N, simulated option value, confidence interval, and difference between 2 trading strategies are



(171383,
 7.187378729746512,
 array([7.13741945, 7.23733801]),
 -0.07068168040061806)